In [1]:
from ctypes import *
from ctypes.wintypes import *
import numpy as np
import psutil
import sys
from util.player import player

In [2]:
NAME_OF_PROCESS = "fm.exe"
PROCESS_ID = None

PLAYERS_START_ADDR = None

PLAYER_ADDR_STRLEN = 4

PROCESS_VM_READ = 0x0010

PLAYER_CLASS_ID = 0x0133f1a80133ec44

CURRENT_DATE = None # (year, days)

KNOWN_PLAYERS_FILE_NAME = "known.slf"
INTERESTED_PLAYERS_FILE_NAME = "interested.slf"

In [3]:
k32 = WinDLL('kernel32')
k32.OpenProcess.argtypes = DWORD,BOOL,DWORD
k32.OpenProcess.restype = HANDLE
k32.ReadProcessMemory.argtypes = HANDLE,LPVOID,LPVOID,c_size_t,POINTER(c_size_t)
k32.ReadProcessMemory.restype = BOOL

In [4]:
def set_pid():
    global PROCESS_ID
    for proc in psutil.process_iter():
        if proc.name() == NAME_OF_PROCESS:
            PROCESS_ID = proc.pid
            break
        
def set_current_date():
    
    global CURRENT_DATE
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(4)
    s = c_size_t()
    
    current_date_addr = 0x0163869c
    
    if k32.ReadProcessMemory(process, current_date_addr, buf, 4, byref(s)):
        reverse_array(buf)
        year = int(buf.raw.hex()[0:4],16)
        days = int(buf.raw.hex()[4:8],16)
        CURRENT_DATE = (year, days)
    else:
        print("set_current_date: Access Denied!")
        
def set_players_start_address():
    
    global PLAYERS_START_ADDRESS
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(4)
    s = c_size_t()
    
    players_start_addr_addr = 0x0183ff10
    
    if k32.ReadProcessMemory(process, players_start_addr_addr, buf, 4, byref(s)):
        reverse_array(buf)
        PLAYERS_START_ADDRESS = int(buf.raw.hex(),16)
    else:
        print("set_players_start_address: Access Denied!")

def reverse_array(array):
    length = len(array)
    left = 0
    right = length-1
    while left < right:
        temp = array[left]
        array[left] = array[right]
        array[right] = temp
        left = left + 1
        right = right - 1

        
def reverse_hex_string(st):
    length = len(st)
    st = list(st)
    left = 0
    right = length
    while left < right:
        temp = st[left:left+2]
        st[left:left+2] = st[right-2:right]
        st[right-2:right] = temp
        left = left + 2
        right = right - 2
    return "".join(st)
        
def decode_player(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(PLAYER_ADDR_STRLEN)
    s = c_size_t()
    
    uid_addr = addr + 232
    
    if k32.ReadProcessMemory(process, uid_addr, buf, 4, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1
    
def decode_player_class_id(addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    buf = create_string_buffer(8)
    s = c_size_t()
    
    if k32.ReadProcessMemory(process, addr, buf, 8, byref(s)):
        reverse_array(buf)
        value = int(buf.raw.hex(),16)
        return value
    else:
        print("Access Denied!")
        return -1

def get_name(name_addr):
    
    process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
    
    buf = create_string_buffer(4)
    s = c_size_t()
    if k32.ReadProcessMemory(process, name_addr, buf, 4, byref(s)):
        reverse_array(buf)
        link1_addr = int(buf.raw.hex(),16) + 0x24
    else:
        print("get_name_point_1: Access Denied!")
        return
    
    buf = create_string_buffer(4)
    s = c_size_t()
    if k32.ReadProcessMemory(process, link1_addr, buf, 4, byref(s)):
        reverse_array(buf)
        link2_addr = int(buf.raw.hex(),16) + 8
    else:
        print("get_name_point_2: Access Denied!")
        return
    
    buf = create_string_buffer(4)
    s = c_size_t()
    if k32.ReadProcessMemory(process, link2_addr, buf, 4, byref(s)):
        reverse_array(buf)
        link3_addr = int(buf.raw.hex(),16)
    else:
        print("get_name_point_3: Access Denied!")
        return
        
    buf = create_string_buffer(100)
    s = c_size_t()
    if k32.ReadProcessMemory(process, link3_addr, buf, 100, byref(s)):
        name_100 = buf.raw.hex()
        name = decode_name(name_100)
        return name
    else:
        print("get_name_point_4: Access Denied!")
        return
    
def decode_name(hex_string):

    name = ""
    start = 0
    
    while True:
        
        hex_segment = hex_string[start:start+4]
        if hex_segment == '0000':
            break
        hex_segment_reversed = reverse_hex_string(hex_segment)
        char = chr(int(hex_segment_reversed,16))
        name = name + char
        start += 4
        
    return name

In [5]:
set_pid()
set_current_date()
set_players_start_address()

In [6]:
players = []

process = k32.OpenProcess(PROCESS_VM_READ, False, PROCESS_ID)
buf = create_string_buffer(PLAYER_ADDR_STRLEN)
s = c_size_t()

addr = PLAYERS_START_ADDRESS

number_of_players = 0
number_of_others = 0

while True:
    
    if k32.ReadProcessMemory(process, addr, buf, PLAYER_ADDR_STRLEN, byref(s)):
        reverse_array(buf)
        player_addr = int(buf.raw.hex(),16)
        if player_addr == 0:
            break
        class_id = decode_player_class_id(player_addr)
        if class_id == PLAYER_CLASS_ID:
            number_of_players += 1
            p = player(PROCESS_ID)
            p.populate_player_from_addr(player_addr)
            players.append(p)
        elif class_id == -1:
            print("Returned -1, breaking")
            break
        else:
            # print(class_id)
            number_of_others += 1
        addr = addr + PLAYER_ADDR_STRLEN
    else:
        print("Access Denied!")
        break

print(number_of_players)
print(number_of_others)

27087
224


In [7]:
for p in players:
    p.set_age(CURRENT_DATE)

In [8]:
known_players_uid = []
interested_players_uid = []
hexdata_known = None
hexdata_interested = None

# Not full-proof as file may be deleted between check and open
try:
    with open(KNOWN_PLAYERS_FILE_NAME, 'rb') as f:
        hexdata_known = f.read().hex()
except Exception as e:
    print("File " + KNOWN_PLAYERS_FILE_NAME + " does not exist", e)

if hexdata_known is not None:
    
    l = len(hexdata_known)
    end = l - 2
    start = end - 8
    
    while True:
        
        hex_string = hexdata_known[start:end]
        if hex_string[4:8] == '0000':
            break
        hex_string_reversed = reverse_hex_string(hex_string)
        known_players_uid.append(int(hex_string_reversed, 16))
        
        start -= 8
        end -= 8
        
        if start < 0:
            break
            
    for p in players:
        if p.uid in known_players_uid:
            p.is_known = True
        else:
            p.is_known = False

# Not full-proof as file may be deleted between check and open
try:
    with open(INTERESTED_PLAYERS_FILE_NAME, 'rb') as f:
        hexdata_interested = f.read().hex()
except Exception as e:
    print("File " + INTERESTED_PLAYERS_FILE_NAME + " does not exist", e)

if hexdata_interested is not None:
    
    l = len(hexdata_interested)
    end = l - 2
    start = end - 8
    
    while True:
        
        hex_string = hexdata_interested[start:end]
        if hex_string[4:8] == '0000':
            break
        hex_string_reversed = reverse_hex_string(hex_string)
        interested_players_uid.append(int(hex_string_reversed, 16))
        
        start -= 8
        end -= 8
        
        if start < 0:
            break
            
    for p in players:
        if p.uid in interested_players_uid:
            p.is_interested = True
        else:
            p.is_interested = False

File known.slf does not exist [Errno 2] No such file or directory: 'known.slf'


In [9]:
players.sort(key=lambda x: x.total, reverse = True)

In [20]:
p = players[5]

In [21]:
p.first_name

'Arne'

In [22]:
p.second_name

'Zimmermann'

In [23]:
p.date_of_birth

(2006, 274)

In [24]:
p.age

(24, 256)

In [25]:
CURRENT_DATE

(2031, 165)

In [26]:
p.club_uid

676

In [27]:
p.is_known

In [28]:
p.is_interested

False

In [29]:
p.club_name

'Liverpool'